# Imports

In [373]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn import svm

# Settings

In [374]:
root_path = os.path.join(os.getcwd(), 'motionsense-dataset')
subjects_info_path = os.path.join(root_path, 'data_subjects_info.csv')
data_root_path = os.path.join(root_path, 'A_DeviceMotion_data')
num_participants = 24

# Data Preparation

In [375]:
def dataframe_from_dir(num_participants, root_path, dir_path):
    '''
    Receives a single experiment dirname
    and returns a list of dataframes for each subject
    of the specified experiment
    '''
    dfs = []
    for i in range(1, num_participants + 1):
        file_name = 'sub_' + str(i) + '.csv'
        file_path = os.path.join(root_path, dir_path, file_name)
        df = pd.read_csv(file_path)
        dfs.append(df)
    return dfs

In [376]:
def dataframes_from_files(num_participants, root_path, data_dirs):
    '''
    Receives a list of directories
    and returns a list of dataframes for each subject
    and each experiment specified
    '''
    dfs = []
    for dir_path in data_dirs:
        dir_dfs = dataframe_from_dir(num_participants, root_path, dir_path)
        dfs.extend(dir_dfs)
    return dfs       

In [377]:
def dataframes_to_feature_vectors(dfs, feature, num_cols):
    '''
    Receives a list of dataframes and a feature
    and returns a dataframe and a matrix
    where each column is a timestamp.
    Note that the number of columns (i.e.) timestamps
    is constant, and should refer to the minimal experiment.
    '''
    data_matrix = []
    
    for df in dfs:
        values = df[feature].head(num_cols).tolist()
        data_matrix.append(values)
    
    feature_df = pd.DataFrame(data_matrix)
    return feature_df, data_matrix

# Sit vs. Walk

In [378]:
train_dirs = ['sit_5', 'sit_13', 'wlk_7', 'wlk_8']
test_dirs = ['wlk_15']
train_labels = 48 * [0]
train_labels.extend(48 * [1])
test_labels = 24 * [1]

In [379]:
train_dfs = dataframes_from_files(num_participants, data_root_path, train_dirs)
test_dfs = dataframes_from_files(num_participants, data_root_path, test_dirs)
dfs = train_dfs + test_dfs
num_cols = min(df.shape[0] for df in dfs) # Cut to minimum experiment length
features = dfs[0].columns.tolist()[1:]

### SVM

In [381]:
def run_SVM(train_set, train_labels, test_set, test_labels):
    '''
    Returns the 0-1 loss and the predicted labels
    '''
    
    classifier = svm.SVC()
    classifier.fit(train_set, train_labels)
    
    predicted_labels = []
    for sample in test_set:
        predicted_labels.append(classifier.predict([sample])[0])
    
    loss = sum(abs(prediction - label) for prediction, label in zip(predicted_labels, test_labels))
    loss /= len(test_labels)
    
    return predicted_labels, loss

### Run SVM for all features

In [382]:
scores = dict()

for feature in features:
    train_feature_df, train_set = dataframes_to_feature_vectors(train_dfs, feature, num_cols)
    test_feature_df, test_set = dataframes_to_feature_vectors(test_dfs, feature, num_cols)
    
    predictions, loss = run_SVM(train_set, train_labels, test_set, test_labels)
    scores[feature] = 1 - loss

In [383]:
scores

{'attitude.pitch': 0.9583333333333334,
 'attitude.roll': 0.7916666666666666,
 'attitude.yaw': 0.625,
 'gravity.x': 0.8333333333333334,
 'gravity.y': 0.9583333333333334,
 'gravity.z': 1.0,
 'rotationRate.x': 1.0,
 'rotationRate.y': 1.0,
 'rotationRate.z': 0.875,
 'userAcceleration.x': 0.20833333333333337,
 'userAcceleration.y': 0.9583333333333334,
 'userAcceleration.z': 0.625}

This is clearly not a clean result since we only test on 24 same-label samples.

# Combine all experiments

In [384]:
def get_experiment_label(dir_name, experiment_labels):
    return experiment_labels[dir_name[:3]]

In [385]:
all_dirs = [dir for dir in os.walk(data_root_path)][0][1]
experiment_labels = {
    'sit': 0,
    'dws': 1,
    'jog': 2,
    'std': 3,
    'ups': 4,
    'wlk': 5
}

# Deep learning solutions

In [388]:
import tensorflow as tf